In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression #logistic回歸
from sklearn.svm import LinearSVC #線性SVM
from skimage.feature import local_binary_pattern
from sklearn import svm, multiclass, model_selection
from sklearn.model_selection import train_test_split
from skimage import io, transform, color, measure, segmentation, morphology, feature
from sklearn.decomposition import PCA 

In [ ]:
def loadimage(n):
    x,y=[],[] 
    n=open(n)
    line=n.readlines()
    
    for i in range(len(line)):
        filename,labs=line[i].split(" ") #用空白分開
        x.append(process(filename))#在一個一個放到陣列裡面 回傳陣列方法
        y.append(int(labs))
        print (i)
    return x,y

In [ ]:
def process(filename):
    image=cv2.imread(filename)
    image=cv2.resize(image,(256,256))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #轉成灰階加快檢測速度
    #lbp = local_binary_pattern(image, n_points, radius)
    return image

In [ ]:
x,y=loadimage("train.txt")
valx,valy=loadimage("val.txt")

In [ ]:
# 填充空白区域
def imfill(im_th):
    # im_th 是0 1 整形二值图
    # Copy the thresholded imapge.
    im_th = np.uint8(im_th)
    im_floodfill = im_th.copy()
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    # print('h '+str(h)+'   w '+str(w))
    mask = np.zeros((h + 2, w + 2), np.uint8)
    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0, 0), 1)
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    return im_out
 

In [ ]:
 # 图片样本进行预处理，进行裁剪，去除非必要部分
vidHeight = 256
vidWidth = 256
# for i in range(0, len(pic_path)):
def imgprocessing(x):
    Data=[]

    for i in range(len(x)):

        thresh = 140
        bw = (x[i] <= thresh/255) * 1  # 根据阈值进行分割
        # dst=np.uint8(dst)
        pic_temp2 = imfill(bw)

        cleared = pic_temp2.copy()  # 复制
        segmentation.clear_border(cleared)  # 清除与边界相连的目标物
        label_image = measure.label(cleared)  # 连通区域标记  connectivity=1 # 4连通区域标记
        # image_label_overlay = color.label2rgb(label_image)  # 不同标记用不同颜色显示
        # plt.imshow(image_label_overlay, interpolation='nearest')
        # plt.show()
        borders = np.logical_xor(bw, cleared)  # 异或,去除背景
        label_image[borders] = -1
        Eccentricity = 1   # 离心率
        for region in measure.regionprops(label_image):  # 循环得到每一个连通区域属性集
            # 忽略小区域

            if region.area < 100000:
                continue
            # print('area is ' + str(region.area) + '  ecc is' + str(region.eccentricity))
            if Eccentricity > region.eccentricity:
                Eccentricity = region.eccentricity
                minr, minc, maxr, maxc = region.bbox  # 绘制外包矩形
        # 判断是否有符合条件的区域
        if 'minr' in vars():
            pic = x[i][minr:maxr, minc:maxc,:]
            pic = transform.resize(pic, [256,256,3])
            #print(lbp)
            # plt.imshow(pic)
            # plt.show()
        else:
            pic = transform.resize(x[i], [256, 256, 3])

        #提取LBP特征，每个图像分成4块进行提取
        pic1 = color.rgb2gray(pic)
        rows, cols = pic1.shape
        radius = 2;
        n_points = radius * 8

        lbp_sum=[]
        for row in range(2):
            for col in range(2):
                #print(str((row * rows//2)) + ' : ' + str(((row+1) * rows//2 - 1)))
                pic1_block = pic1[(row * rows//2) : ((row+1) * rows//2 - 1) , (col * col//2) : ((col+1) * col//2 - 1)]
                lbp = feature.local_binary_pattern(pic1, n_points, radius, 'uniform')
                lbp2 = lbp.astype(np.int32)
                max_bins = int(lbp2.max() + 1)
                train_hist, _ = np.histogram(lbp2, normed=True, bins=max_bins, range=(0, max_bins))
               # print(train_hist.dtype)
                #print(train_hist)
                lbp_sum=lbp_sum + train_hist.tolist()

                #
        print(i)



        Data.append(lbp_sum)
    return Data
#使用SVM进行训练并计算测试准确率
print ('almost')

In [ ]:
#print (y_train[5:50])
#print (len(X_train))

In [ ]:
datax=imgprocessing(x)
datavalx=imgprocessing(valx)

In [ ]:

# train_data = Data[0:7]
# train_label = label1[0:7]
# test_data = Data[8:9]
# test_label = label1[8:9]
svr_rbf = svm.SVR(kernel='rbf', C=1e3, gamma=0.1);
model = multiclass .OneVsRestClassifier(svr_rbf,-1)  #.fit(train_data, train_label).score(test_data,test_label)
clf = model.fit(datax, y)
sore=clf.score(datavalx, valy)
print('acc'+str(sore))